In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

import os

import march_madness as mm
import random

In [2]:
def get_win_location_weight(loc):
    if loc == 'H':
        return 0.6
    elif loc == 'A':
        return 1.4
    else:
        return 1

def get_loss_location_weight(loc):
    if loc == 'A':
        return 1.4
    elif loc == 'H':
        return 0.6
    else:
        return 1

In [3]:
analysis = mm.Analysis()

In [4]:
analysis.load_season(1985)

In [ ]:
analysis.calc_seasons_features()

In [5]:
season = analysis.seasons[1985]

In [6]:
season.regular_season_results

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WFGM,WFGA,...,LFGA3,LFTM,LFTA,LOR,LDR,LAst,LTO,LStl,LBlk,LPF


In [ ]:
season.tourney_seeds

In [ ]:
team = season.teams[1261]

In [ ]:
team.win_data

In [ ]:
team.loss_data

In [ ]:
team.win_data.shape

In [ ]:
team.features.keys()

In [ ]:
season.calc_teams_net_detailed_game_stats()

In [ ]:
team.features.keys()

metric_keys = ['FGM','FGA','FGM3','FGA3','FTM','FTA','OR','DR','Ast','TO','Stl','Blk','PF',
               'TR','FGM2','FGA2','FG%','FG2%','FG3%','FGA3%','FT%','Pos','OEff']

### Team Net Stats

precision = 4
for metric in metric_keys:
    net_values = []
    game_count = 0
    for i in team.win_data.index:
        game = team.win_data.loc[i]
        
        team_game_value = game[f'W{metric}']
        
        opp_id = game['LTeamID']
        opp_avg_value = season.teams[opp_id].features[f'Opp Avg {metric}']
        
        if np.isnan(team_game_value) or np.isnan(opp_avg_value):
            continue
        
        net_value = team_game_value - opp_avg_value
        
        net_values.append(net_value)
        
    for i in team.loss_data.index:
        game = team.loss_data.loc[i]
        
        team_game_value = game[f'L{metric}']
        
        opp_id = game['WTeamID']
        opp_avg_value = season.teams[opp_id].features[f'Opp Avg {metric}']
        
        if np.isnan(team_game_value) or np.isnan(opp_avg_value):
            continue
        
        net_value = team_game_value - opp_avg_value
        
        net_values.append(net_value)
        
    team_net_value = np.mean(net_values)
    team.features[f'Net Team Avg {metric}'] = round(team_net_value,precision)

for metric in metric_keys:
    net_values = []
    game_count = 0
    for i in team.win_data.index:
        game = team.win_data.loc[i]
        
        opp_game_value = game[f'L{metric}']
        
        opp_id = game['LTeamID']
        opp_avg_value = season.teams[opp_id].features[f'Team Avg {metric}']
        
        if np.isnan(opp_game_value) or np.isnan(opp_avg_value):
            continue
        
        net_value = opp_game_value - opp_avg_value
        
        net_values.append(net_value)
        
    for i in team.loss_data.index:
        game = team.loss_data.loc[i]
        
        opp_game_value = game[f'W{metric}']
        
        opp_id = game['WTeamID']
        opp_avg_value = season.teams[opp_id].features[f'Team Avg {metric}']
        
        if np.isnan(opp_game_value) or np.isnan(opp_avg_value):
            continue
        
        net_value = opp_game_value - opp_avg_value
        
        net_values.append(net_value)
        
    team_net_value = np.mean(net_values)
    team.features[f'Net Opp Avg {metric}'] = round(team_net_value,precision)

team.features.keys()